# 09: Create compiled datasets and summary statistics
*Compile datasets from different GCMs and create summary statistics (e.g., annual maxima, days over threshold).*

In [ ]:
import coiled
import dask
import xarray as xr
from utils import gcm_list, load_multimodel_results, summarize

Set up cluster to handle multiprocessing using a Dask client.

In [ ]:
cluster = coiled.Cluster(
    n_workers=2,
)
client = cluster.get_client()

Compile all individual GCM datasets into one multimodel dataset that is optimally chunked for timeseries analysis. Create summaries for each analysis period and multimodel medians of those summary statistics.

In [ ]:
rename_dict = {"wbgt-shade": "scen", "wbgt-sun": "WBGT-sun"}
analysis_period = {
    "historical": slice("1985", "2014"),
    "ssp245-2030": slice("2020", "2039"),
    "ssp245-2050": slice("2040", "2059"),
    "ssp245-2070": slice("2060", "2079"),
    "ssp370-2030": slice("2020", "2039"),
    "ssp370-2050": slice("2040", "2059"),
    "ssp370-2070": slice("2060", "2079"),
}
scenarios = ["ssp245", "ssp370"]

In [ ]:
for metric in ["wbgt-sun", "wbgt-shade"]:
    full_ds = load_multimodel_results(gcm_list, scenarios, metric)
    full_ds = full_ds.chunk({"time": -1, "processing_id": 850, "gcm": -1})
    full_ds = full_ds.rename({rename_dict[metric]: metric})
    for scenario, timeframe in analysis_period.items():
        compiled_store = (
            "s3://carbonplan-scratch/extreme-heat-extension/v1.0/"
            + f'outputs/zarr/daily/{scenario}-WBGT-{metric.split("-")[1]}.zarr'
        )
        full_ds.sel(time=timeframe).to_zarr(compiled_store, mode="w", consolidated=True)

        ds = xr.open_zarr(compiled_store)
        summarized = summarize(ds[metric], metric.split("-")[0]).load()

        annual_medians = summarized.sel(year=timeframe).median(dim="year")
        ensemble_median = annual_medians.median(dim="gcm")
        results = xr.concat(
            [
                annual_medians,
                ensemble_median.expand_dims(dim={"gcm": ["multimodel_median"]}),
            ],
            dim="gcm",
        ).load()
        summary_store = (
            "s3://carbonplan-scratch/extreme-heat/v1.0/outputs/"
            + f'zarr/summaries/{scenario}-summaries-WBGT-{metric.split("-")[1]}.zarr'
        )
        print(summary_store)
        results = dask.optimize(results)[0]
        results.to_zarr(summary_store, mode="w", consolidated=True)